## Data Wrangling 

## Introduction

This step focus on collecting the data, organizing it and making sure it's well defined.

### Recap Of Data Science Problem

The purpose of this data science project is to come up with a delinquency prediction model for lending club. Decreasing the delinquency would help to increase revenue for lending club and attract more inventor for the platform. The project aims to build a predictive model based on user(borrower) features,which could be categorized into as following: 
- User feature (general)
- User feature (financial specific)
- Credit scores
- Credit lines
- Loan general feature
- Loan payment feature

- Potential response variables:sub_grade, int_rate, loan_status


## Import Lending Club Dataset from  2018Q2

### Glance first two rows in the dataset and decide how to import the whole dataset in pandas 

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn import preprocessing

In [2]:
with open(r'D:\Springboard\Github\Lending Club Capstone Project\LoanStats_securev1_2018Q1.csv') as df:
    for i in range(3):
        print(df.readline())

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","fico_range_low","fico_range_high","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","last_fico_range_high","last_fico_range_low","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joi

The first line is comment in the dataset, so it could be neglected, then the dataset can be loaded into jupyter notebook from the second line. 

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/rhodapang/Lending-Club-Capstone-Project/main/LoanStats_securev1_2018Q1.csv',
                 skiprows=1,sep=',')

C:\Users\rhoda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107866 entries, 0 to 107865
Columns: 150 entries, id to settlement_term
dtypes: float64(112), object(38)
memory usage: 123.4+ MB


In [4]:
df.shape

(107866, 150)

In [5]:
df.head() 

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,129897398,NaN,35000.0,35000.0,35000.0,60 months,13.58%,806.79,C,C2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,130827714,NaN,20000.0,20000.0,20000.0,36 months,5.31%,602.21,A,A1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,130910225,NaN,7000.0,7000.0,7000.0,36 months,11.98%,232.44,B,B5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,130950844,NaN,20000.0,20000.0,20000.0,60 months,9.92%,424.16,B,B2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,130960835,NaN,7000.0,7000.0,7000.0,36 months,20.39%,261.54,D,D4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.dtypes.unique()

array([dtype('O'), dtype('float64')], dtype=object)

## Prelimenarily explore data

### Data cleaning- Dropping irrelevant columns 

The purpose of this data science project is to build a predictive model to identify loans with delinquency status, therefore, some of columns that are related to outcome variables could be dropped in the beginning. From the document-LCDataDictionary provided by lending club, there are total categories variables in the dataset:
- User feature (general)
- User feature (financial specific): income, credit scores, credit lines
- Loan general feature
- Loan payment feature
- Current loan payment feature
- Secondary application info
- Hardship
- Settlement

- Potential response variables:sub_grade, int_rate, loan_status

Generally Speaking, the columns that have not been located in User feature (general),User feature (financial specific),Secondary application info and Loan general feature would be excluded in the dataset.

Because the response variabe in the model is loan_status, the features categories should be included in the model are User feature (general),User feature (financial specific), Secondary application info 

### From the Lending Club Data Dictionary, we know which columns belongs to which categories

In [5]:
# the columns are about Current loan payment feature
Current_loan_payment_feature = ['last_pymnt_amnt','last_pymnt_d','next_pymnt_d','pymnt_plan','total_pymnt','total_pymnt_inv',
                                'total_rec_int','total_rec_late_fee','total_rec_prncp','recoveries','collection_recovery_fee',
                                'out_prncp','out_prncp_inv']
# the columns are about Hardship 
Hardship = ['hardship_flag','hardship_type','hardship_reason','hardship_status','deferral_term','hardship_amount', 'hardship_start_date'
            ,'hardship_end_date','payment_plan_start_date','hardship_length','hardship_dpd','hardship_loan_status',
            'orig_projected_additional_accrued_interest','hardship_payoff_balance_amount','hardship_last_payment_amount']

# the columns are about Settlement
Settlement = ['debt_settlement_flag','debt_settlement_flag_date','settlement_status','settlement_date','settlement_amount',
              'settlement_percentage','settlement_term']

#### Drop above three categorical columns 

In [6]:
for i in [Current_loan_payment_feature,Hardship,Settlement]:
    for j in i:
        df.drop(j,axis=1, inplace=True)

In [7]:
# After dropping the unrelavant columns, there are total 115 columns in the dataset
df.shape

(107866, 115)

#### Additionally, the column of ID, url, issue_d are irrelevant to the model building, so they could be dropped, too. 
- ID:A unique LC assigned ID for the loan listing.
- issue_d:The month which the loan was funded
- url: URL for the LC page with listing data.

In [8]:
df.drop(columns=['id','url','issue_d'],inplace= True)

In [9]:
# Until now, there are 112 columns in the dataset
df.shape

(107866, 112)

### Data cleaning- Missing Value

Because of the large dataset with 112 columns, it'd better to have a big picture that missing value consist in the dataset. 

####  Build a missing table for better understand the dataset 

In [10]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_table = missing_values_table(df)
missing_table

Your selected dataframe has 112 columns.
There are 112 columns that have missing values.


,Missing Values,% of Total Values
member_id,107866,100.0
desc,107866,100.0
sec_app_mths_since_last_major_derog,102439,95.0
mths_since_last_record,92597,85.8
verification_status_joint,91849,85.2
...,...,...
application_type,2,0.0
policy_code,2,0.0
collections_12_mths_ex_med,2,0.0
last_fico_range_low,2,0.0


In [11]:
df_m = missing_table[missing_table['% of Total Values']>=80]
df_m

,Missing Values,% of Total Values
member_id,107866,100.0
desc,107866,100.0
sec_app_mths_since_last_major_derog,102439,95.0
mths_since_last_record,92597,85.8
verification_status_joint,91849,85.2
sec_app_revol_util,91845,85.1
sec_app_mort_acc,91535,84.9
dti_joint,91535,84.9
annual_inc_joint,91535,84.9
revol_bal_joint,91535,84.9


####  Before understand the deep meaning of all columns, the columns with all missing value could be dropped firstly.

In [12]:
df.dropna(axis = 1, how='all', inplace=True)
df.shape

(107866, 110)

In [13]:
# drop the rows that are all missing value, so there have been two rows dropped. 
df = df.dropna(how='all')
df.shape 

(107864, 110)

### Drop columns with only containing the distinct value

In [14]:
drop_cols = [c for c in list(df) if df[c].nunique()<=1]
drop_cols

['policy_code', 'num_tl_120dpd_2m']

In [15]:
df.drop([ 'policy_code', 'num_tl_120dpd_2m'],axis =1,inplace=True)

### drop the deplicated rows in dataframe

In [16]:
# Select all duplicate rows based on all columns and it found out that no row that are duplicated   
df_duplicate = df[df.duplicated(keep = False)]
df_duplicate

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog


### Check the correlations between these variables.

Explore weather there are some columns which are highly multicolinearity,if yes, it could be dropped one of them due to the lots of columns existing.

In [ ]:
# Generally, the default correlation coefficent of .corr() method is the Pearson correlation coefficient, which shows us how close to a 
# straight  line the data-point fall and is a number between -1 and 1. 

In [17]:
# At current stage, the high correlation variables should be paid attention.  
corr_matrix = df.corr().abs()
sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))

In [18]:
sol_df = pd.DataFrame(sol)

In [19]:
# Set the two indexes into columns
sol_df.reset_index(inplace=True)

In [20]:
# rename each column 
sol_df.rename(columns={'level_0':'var_1','level_1':'var_2',0:'cor_coef'},inplace=True)

In [27]:
sol_df

,var_1,var_2,cor_coef
0,loan_amnt,funded_amnt,1.000000
1,sec_app_fico_range_low,sec_app_fico_range_high,1.000000
2,fico_range_low,fico_range_high,1.000000
3,loan_amnt,funded_amnt_inv,0.999996
4,funded_amnt,funded_amnt_inv,0.999996
...,...,...,...
3795,tot_coll_amt,open_il_12m,0.000052
3796,total_cu_tl,bc_util,0.000045
3797,installment,tot_coll_amt,0.000035
3798,delinq_amnt,num_actv_bc_tl,0.000031


#### Select all columns that their value is greater 80%.

High Correlation Coefficients
Pairwise correlations among independent variables might be high (in absolute value).
Rule of thumb: If the correlation > 0.8 then severe multicollinearity may be present. 

In [25]:
high_cor = sol_df[sol_df['cor_coef']>= 0.8]
high_cor

,var_1,var_2,cor_coef
0,loan_amnt,funded_amnt,1.000000
1,sec_app_fico_range_low,sec_app_fico_range_high,1.000000
2,fico_range_low,fico_range_high,1.000000
3,loan_amnt,funded_amnt_inv,0.999996
4,funded_amnt,funded_amnt_inv,0.999996
5,open_acc,num_sats,0.999464
6,num_actv_rev_tl,num_rev_tl_bal_gt_0,0.985972
7,tot_cur_bal,tot_hi_cred_lim,0.981144
8,total_bal_il,total_il_high_credit_limit,0.949234
9,loan_amnt,installment,0.944942


In [27]:
# Calculate the number of each columns and drop the columns with more missing value
high_cor['var_1_miss'] = [df[i].isnull().sum() for i in high_cor['var_1']]
high_cor['var_2_miss'] = [df[i].isnull().sum() for i in high_cor['var_2']]

C:\Users\rhoda\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\rhoda\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


From below table, the value in var_1_miss less or equal than var_2_miss, except 'mths_since_recent_bc_dlq' and 'percent_bc_gt_75'

In [36]:
high_cor

,var_1,var_2,cor_coef,var_1_miss,var_2_miss
0,loan_amnt,funded_amnt,1.000000,0,0
1,sec_app_fico_range_low,sec_app_fico_range_high,1.000000,91533,91533
2,fico_range_low,fico_range_high,1.000000,0,0
3,loan_amnt,funded_amnt_inv,0.999996,0,0
4,funded_amnt,funded_amnt_inv,0.999996,0,0
5,open_acc,num_sats,0.999464,0,0
6,num_actv_rev_tl,num_rev_tl_bal_gt_0,0.985972,0,0
7,tot_cur_bal,tot_hi_cred_lim,0.981144,0,0
8,total_bal_il,total_il_high_credit_limit,0.949234,0,0
9,loan_amnt,installment,0.944942,0,0


In [47]:
# I decide to drop the variabales in var_2, except 'mths_since_recent_revol_delinq'
col_drop=high_cor['var_2'].unique()
col_drop = col_drop.tolist()
col_drop.remove('mths_since_recent_revol_delinq') 
col_drop

['funded_amnt',
 'sec_app_fico_range_high',
 'fico_range_high',
 'funded_amnt_inv',
 'num_sats',
 'num_rev_tl_bal_gt_0',
 'tot_hi_cred_lim',
 'total_il_high_credit_limit',
 'installment',
 'num_tl_30dpd',
 'total_bal_ex_mort',
 'total_bc_limit',
 'num_op_rev_tl',
 'last_fico_range_low',
 'num_actv_rev_tl',
 'acc_open_past_24mths',
 'num_rev_accts',
 'percent_bc_gt_75',
 'num_tl_op_past_12m',
 'num_bc_sats',
 'avg_cur_bal',
 'num_bc_tl']

In [49]:
for i in col_drop:
    df.drop(i,axis=1, inplace=True)

In [ ]:
df = df.drop(columns = 'mths_since_recent_bc_dlq')

In [52]:
df.shape

(107864, 85)

In [53]:
# after preliminarily drop those high corelated variables, we see the high_cor table again. 
corr_matrix = df.corr().abs()
sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))
sol_df = pd.DataFrame(sol)
# Set the two indexes into columns
sol_df.reset_index(inplace=True)
# rename each column 
sol_df.rename(columns={'level_0':'var_1','level_1':'var_2',0:'cor_coef'},inplace=True)
high_cor = sol_df[sol_df['cor_coef']>= 0.8]
high_cor

,var_1,var_2,cor_coef
0,mths_since_last_delinq,mths_since_recent_revol_delinq,0.860612


In [56]:
df['mths_since_last_delinq'].isnull().sum(), df['mths_since_recent_revol_delinq'].isnull().sum()

(60695, 77038)

In [57]:
# drop variable 'mths_since_recent_revol_delinq' due to more missing value 
df = df.drop(columns = 'mths_since_recent_revol_delinq')

In [59]:
# After finishing the preliminarily explore, there are 108 columns in the dataset 

df.shape

(107864, 84)

In [60]:
# Save the dataframe for next stage data analysis
df.to_csv(r'D:\Springboard\Github\Lending Club Capstone Project\Lending Club Updated Version.csv',index=False)